In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [9]:
# API Call
load_dotenv()
API_KEY = os.getenv("yelp_api") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"


In [10]:
print(API_KEY)

None


In [6]:
# Test call
#search_limit_test = 10
#radius_test = 2000 #radius in meters
#term_test = "restaurants"
#location_test = "Atlanta" #use to test the json response
#search_params_test = {"location": location_test, "term": term_test, "limit": search_limit_test, "radius": radius_test,
                 #"sort_by": "rating"}
#response_test = requests.get(url = base_url, params = search_params_test, headers = headers).json()
#pprint(response_test)

In [7]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204


In [8]:
# Make a list of coordinate tuples (lat, lng) for each stadium
stadium_coordinates = list(zip(stadium_data_df["Latitude"], stadium_data_df["Longitude"]))
print(stadium_coordinates)

[(33.528, -112.26299999999999), (33.755556, -84.4), (39.278056, -76.622778), (42.773999999999994, -78.78699999999999), (35.225833, -80.852778), (41.8623, -87.6167), (39.095, -84.516), (41.506111, -81.699444), (32.747778000000004, -97.09277800000001), (39.741, -105.0197), (42.34, -83.045556), (44.501389, -88.06222199999999), (29.684721999999997, -95.410833), (39.760056, -86.16380600000001), (30.323889, -81.6375), (39.048889, -94.483889), (36.090833, -115.18361100000001), (33.95345, -118.3392), (33.95345, -118.3392), (25.958056, -80.238889), (44.974, -93.258), (42.091, -71.264), (29.950833000000003, -90.08111099999999), (40.813528000000005, -74.074361), (40.813528000000005, -74.074361), (39.900833, -75.1675), (40.446667, -80.0158333), (37.403, -121.97), (47.5952, -122.3316), (27.975833, -82.503333), (36.166389, -86.771389), (38.90466305, -76.85866323)]


In [9]:
# Loop to search restaurants around the stadiums
search_limit = 50 #imit to 50 business names per query
radius = 3000 #radius in meters
term = "restaurants"
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating"}
#search_params = {"term": term, "radius": radius, "sort_by": "rating"}
# lists to store the data retrieve from Yelp
business_name = []
business_city = []
business_lat = []
business_lng = []
business_price = []
business_rating = []
business_reviews = []
number_business = []
# Dictionary to create a dataframe from the api request
business_dict = {"Name": business_name, "City": business_city, "Price": business_price, 
                 "Rating": business_rating, "Number of reviews": business_reviews, 
                 "Latitude": business_lat, "Longitude": business_lng}
# list of coordinates to use to test the for loop.
coordinates_test = [(33.528, -112.26299999999999), (33.755556, -84.4)]

In [10]:
# Loop to retrieve the data based on (lat,lng)
for latitude, longitude in stadium_coordinates:
    #print(f"lat = {latitude}")
    #print(f"lng = {longitude}")
    
    search_params.update({"latitude": latitude, "longitude": longitude})
    response = requests.get(url = base_url, params = search_params, headers = headers).json()
    
    number_business.append(response["total"])
    
    #search_params.update({"limit": limite}) 
    #pprint(response)
    #for i in range(search_limit):
        #business_name.append(response['businesses'][i]['name'])
        
        #if search_limit< limite:
            #search_limit +=50
        #else:
            #print("error")

            
    

KeyError: 'total'

In [10]:
print(number_business)

[94, 706, 665, 51, 442, 1400, 458, 354, 210, 573, 531, 120, 180, 355, 174, 17, 756, 293, 293, 84, 682, 44, 945, 133, 133, 395, 380, 181, 1400, 269, 528, 94]


In [11]:
#Add number of businesses to the data frame
stadium_data_df["Number of restaurants"]=number_business

In [12]:
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code,Number of restaurants
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305,94
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313,706
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230,665
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127,51
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202,442
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605,1400
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202,458
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114,354
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011,210
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204,573


In [15]:
# 15000 meters (10 miles) radius
big_radius = 15000 #radius in meters
search_params_big = {"term": term, "limit": search_limit, "radius": big_radius, "sort_by": "rating"}
big_number_business = []
for latitude, longitude in stadium_coordinates:
        
    search_params_big.update({"latitude": latitude, "longitude": longitude})
    response2 = requests.get(url = base_url, params = search_params_big, headers = headers).json()
    
    big_number_business.append(response2["total"])

In [16]:
print(big_number_business)

[1900, 3600, 2900, 1300, 2100, 6400, 1700, 1600, 2100, 3800, 2300, 443, 3900, 2000, 1700, 1500, 4500, 10600, 10600, 3800, 2600, 637, 2200, 18300, 18300, 5100, 2100, 3900, 4200, 2000, 2000, 1500]


In [11]:
pprint(response)

NameError: name 'response' is not defined